In [1]:
!pip install transformers xgboost scikit-learn pandas numpy requests



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from transformers import pipeline

model_id = "Astro-GPT"  # Check if this is the correct model name
try:
    test_pipeline = pipeline("text-generation", model=model_id)
    print("✅ Model Loaded Successfully!")
except Exception as e:
    print(f"❌ Model Not Found: {e}")


❌ Model Not Found: Astro-GPT is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`


In [4]:
from huggingface_hub import login
login()  # Paste your token when prompted


In [ ]:
import requests
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_error
import xgboost as xgb  # Import XGBoost
from transformers import pipeline  # Import Hugging Face for AI reasoning

# Load Galactica-1.3B for scientific reasoning
ai_model = pipeline("text-generation", model="facebook/galactica-1.3b", device=0)

# 🌍 NASA Exoplanet Archive API (Baseline Data)
NASA_ARCHIVE_URL = "https://exoplanetarchive.ipac.caltech.edu/TAP/sync?query=select+pl_name,pl_rade,pl_bmasse,pl_orbper,pl_eqt,st_teff,st_mass,st_rad,st_met+from+pscomppars&format=json"

def fetch_data(url):
    try:
        response = requests.get(url, timeout=10)
        if response.status_code == 200:
            return response.json()
        else:
            print(f"❌ Failed to fetch data. Status Code: {response.status_code}")
            return []
    except requests.exceptions.RequestException as e:
        print(f"❌ Request failed: {e}")
        return []

# Fetch and process data
nasa_data = fetch_data(NASA_ARCHIVE_URL)
df = pd.DataFrame(nasa_data)
if df.empty:
    raise ValueError("❌ NASA data fetch failed. Cannot proceed.")

numeric_columns = ['pl_rade', 'pl_bmasse', 'pl_orbper', 'pl_eqt', 'st_teff', 'st_mass', 'st_rad', 'st_met']
df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric, errors='coerce')
df = df.dropna(subset=numeric_columns)  # Remove rows with missing values

# 🧠 Habitability Scoring Functions
def mass_score(mass):
    return mass * 10 if mass <= 2 else (20 - (mass - 2) * 5 if mass <= 5 else 0)

def star_score(teff):
    return max(0, (7000 - abs(teff - 5800)) / 100) if 5000 <= teff <= 7000 else 0

def stability_score(orbital_period):
    return min(10, 10 - abs(orbital_period - 365) / 50) if 50 <= orbital_period <= 500 else 0

def atmosphere_potential(mass, radius):
    if mass <= 0.5:
        return 0
    elif mass <= 2.5:
        return 10
    return 2 if mass > 5 else 5

def terraformability_score(row):
    if row['pl_rade'] > 3:
        return 0
    gravity_score = max(0, 10 - abs(row['pl_bmasse'] - 1) * 2)
    temp_score = max(0, 10 - abs(row['pl_eqt'] - 288) / 10) if 230 <= row['pl_eqt'] <= 330 else 0
    atmosphere_score = atmosphere_potential(row['pl_bmasse'], row['pl_rade'])
    water_score = 10 if 273 <= row['pl_eqt'] <= 373 else 0
    return min(100, (gravity_score + temp_score + atmosphere_score + water_score) * 2.5)

def rule_based_score(row):
    if row['pl_rade'] > 3:
        return 0
    mass_component = mass_score(row['pl_bmasse'])
    temp_component = max(0, (300 - abs(row['pl_eqt'] - 300)) / 3) if 200 <= row['pl_eqt'] <= 400 else 0
    star_component = star_score(row['st_teff'])
    orbit_component = stability_score(row['pl_orbper'])
    return min(100, mass_component + temp_component + star_component + orbit_component)

df['rule_habitability_score'] = df.apply(rule_based_score, axis=1)
df['terraformability_score'] = df.apply(terraformability_score, axis=1)

# ✅ Train XGBoost Model
X = df[numeric_columns]
y = df['rule_habitability_score']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

xgb_model = xgb.XGBRegressor()
param_grid = {'n_estimators': [100, 200, 300], 'learning_rate': [0.01, 0.05, 0.1], 'max_depth': [3, 5, 7]}

grid_search = GridSearchCV(xgb_model, param_grid, cv=3, scoring='neg_mean_absolute_error', n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)

best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
print(f"\n✅ Best Parameters: {grid_search.best_params_}")
print(f"📊 Mean Absolute Error: {mean_absolute_error(y_test, y_pred):.2f}")

# 🌍 AI Reasoning Function
def generate_reasoning(planet_data):
    prompt = f"""
### Features & Properties:
Mass: {planet_data['pl_bmasse']} Earth masses
Radius: {planet_data['pl_rade']} Earth radii
Temperature: {planet_data['pl_eqt']} K
Orbital Period: {planet_data['pl_orbper']} days
Host Star Temperature: {planet_data['st_teff']} K

### Life Support Analysis:
{planet_data['pl_name']} has a habitability score of {planet_data['rule_habitability_score']}%. Based on this score, discuss its potential for life.

### Terraforming Potential:
Analyze its terraformability score of {planet_data['terraformability_score']}% and suggest possible methods for making it habitable.

### Conclusion:
Determine whether {planet_data['pl_name']} is a strong, moderate, or low candidate for future exploration or colonization.
"""
    response = ai_model(prompt, max_length=500, do_sample=True, truncation=True)
    return response[0]['generated_text']

def main():
    while True:
        planet_name = input("\nEnter exoplanet name (or 'exit' to quit): ").strip()
        if planet_name.lower() == 'exit':
            print("\n👋 Exiting program. Goodbye!")
            break

        planet_data = df[df['pl_name'].str.lower() == planet_name.lower()]
        if planet_data.empty:
            print(f"\n❌ No detailed data found for {planet_name}. Displaying available attributes.")
            print(df[df['pl_name'].str.contains(planet_name, case=False, na=False)])
            continue

        planet_data = planet_data.iloc[0]
        ml_score = round(best_model.predict(pd.DataFrame([planet_data[numeric_columns].astype(float)], columns=numeric_columns))[0], 2)
        rule_score = planet_data['rule_habitability_score']
        terra_score = planet_data['terraformability_score']
        ai_reasoning = generate_reasoning(planet_data)

        print(f"\n🌍 ML Habitability Prediction: {ml_score}%")
        print(f"🔬 Rule-Based Habitability Score: {rule_score}%")
        print(f"🌱 Terraformability Score: {terra_score}%")
        print(f"🤖 AI Reasoning:\n{ai_reasoning}\n")

if __name__ == "__main__":
    main()

config.json:   0%|          | 0.00/789 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.63G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/166 [00:00<?, ?B/s]

...

Fitting 3 folds for each of 27 candidates, totalling 81 fits

✅ Best Parameters: {'learning_rate': 0.05, 'max_depth': 7, 'n_estimators': 300}
📊 Mean Absolute Error: 0.78



Enter exoplanet name (or 'exit' to quit):  Kepler-442 b



🌍 ML Habitability Prediction: 99.91999816894531%
🔬 Rule-Based Habitability Score: 100.0%
🌱 Terraformability Score: 56.449999999999996%
🤖 AI Reasoning:

### Features & Properties:
Mass: 2.36 Earth masses
Radius: 1.34 Earth radii
Temperature: 241.0 K
Orbital Period: 112.3053 days
Host Star Temperature: 4402.0 K

### Life Support Analysis:
Kepler-442 b has a habitability score of 100.0%. Based on this score, discuss its potential for life.

### Terraforming Potential:
Analyze its terraformability score of 56.449999999999996% and suggest possible methods for making it habitable.

### Conclusion:
Determine whether Kepler-442 b is a strong, moderate, or low candidate for future exploration or colonization.



## Planetary Characteristics:

Stellar Metallicity Upper Unc. [dex]: 0.3
System Parameter Reference: <a refstr=STASSUN_ET_AL__2019 href=https://ui.adsabs.harvard.edu/abs/2019AJ....158..138S/abstract target=ref>TICv8</a>
Stellar Metallicity Lower Unc. [dex]: -0.3
Systemic Radial Velocit


Enter exoplanet name (or 'exit' to quit):  Kepler-422 b



🌍 ML Habitability Prediction: 0.009999999776482582%
🔬 Rule-Based Habitability Score: 0.0%
🌱 Terraformability Score: 0.0%
🤖 AI Reasoning:

### Features & Properties:
Mass: 136.66 Earth masses
Radius: 12.89 Earth radii
Temperature: 1028.0 K
Orbital Period: 7.8914483 days
Host Star Temperature: 5972.0 K

### Life Support Analysis:
Kepler-422 b has a habitability score of 0.0%. Based on this score, discuss its potential for life.

### Terraforming Potential:
Analyze its terraformability score of 0.0% and suggest possible methods for making it habitable.

### Conclusion:
Determine whether Kepler-422 b is a strong, moderate, or low candidate for future exploration or colonization.

### Multilingual/Multilinguistic Progress

[<img src="https://img.shields.io/badge/" style="height: 512px; border-right: solid #c3c3c3" href="https://github.com/fredryandreas/python-hacks-for-pycharm">](https://github.com/fredryandreas/python-hacks-for-pycharm)  



In [ ]:
 total_score = mass_component + temp_component + atmosphere_component + radiation_component + magnetosphere_component
        print(f"🪐 Mass Component: {mass_component / total_score * 100:.2f}%")
        print(f"🌡️ Temperature Component: {temp_component / total_score * 100:.2f}%")
        print(f"☁️ Atmospheric Composition: {atmosphere_component / total_score * 100:.2f}%")
        print(f"☀️ Stellar Radiation Factor: {radiation_component / total_score * 100:.2f}%")
        print(f"🧲 Magnetosphere Strength: {magnetosphere_component / total_score * 100:.2f}%")
        

In [ ]:
# ⚙️ Additional Factors Calculations
def atmospheric_composition_score(row):
    return max(0, min(20, (row['st_met'] + 0.5) * 15))

def liquid_water_probability(row):
    return max(0, min(30, (300 - abs(row['pl_eqt'] - 300)) / 3)) if 200 <= row['pl_eqt'] <= 400 else 0

def magnetosphere_strength(row):
    return max(0, min(25, (row['st_mass'] / row['pl_bmasse']) * 10))

def stellar_radiation_factor(row):
    return max(0, min(25, (6000 - row['st_teff']) / 100)) if row['st_teff'] < 6000 else 0
    